In [ ]:
from pycocotools.coco import COCO
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as transforms
from PIL import Image
import torchvision.models.detection as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
def resize_boxes(boxes, orig_size, target_size):
    orig_width, orig_height = orig_size
    target_width, target_height = target_size

    # Calcular fatores de escala
    scale_x = target_width / orig_width
    scale_y = target_height / orig_height
    #print(boxes)
    # Ajustar as caixas
    new_boxes = [boxes[0]*scale_x, boxes[1]*scale_y, boxes[2]*scale_x, boxes[3]*scale_y]

    return new_boxes

In [ ]:
from torch.utils.data import Dataset

class COCODataset(Dataset):
    def __init__(self, coco_json_path, images_dir, transform=None, resize=None):
        self.coco = COCO(coco_json_path)
        self.images_dir = images_dir
        self.image_ids = self.coco.getImgIds()
        self.transform = transform
        self.resize = resize

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        image_id = self.image_ids[idx]
        image_data = self.coco.loadImgs(image_id)[0]
        image_path = os.path.join(self.images_dir, image_data['file_name'])

        # Carregar imagem
        image = Image.open(image_path).convert("RGB")
        width, height = image.size

        # Aplicar transformações
        if self.transform:
            image = self.transform(image)

        # Obter anotações
        ann_ids = self.coco.getAnnIds(imgIds=image_id)
        annotations = self.coco.loadAnns(ann_ids)

        boxes = []
        labels = []
        for ann in annotations:
            #print(ann['bbox'])
            if self.resize is not None:
                x, y, w, h = resize_boxes(ann["bbox"], (width, height), self.resize)
            else:
                x, y, w, h = ann["bbox"]
            boxes.append([x, y, x + w, y + h])
            labels.append(ann["category_id"])

        return image, torch.tensor(boxes), torch.tensor(labels)

# Criar dataset
# transform = transforms.Compose([
#     transforms.Resize((300, 300)),
#     transforms.ToTensor(),
# ])
#dataset = COCODataset(coco_json_path, images_dir, transform=transform, resize=(300, 300))


In [ ]:
from torchvision.models.detection.ssd import ssd300_vgg16
from IPython.display import clear_output
from tqdm.auto import tqdm

def treinamento(dataset, num_epochs=150, alias=""):
    # Criar modelo SSD com backbone VGG16
    model = ssd300_vgg16(pretrained=True).to(device)
    model.train()
    
    # Definir otimizador e loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = torch.nn.CrossEntropyLoss()
    
    out_epochs = []
    
    # Loop de treinamento

    clear_count = 0
    clear_rate = 10
    #p_images = tqdm(total=len(dataset))
    for epoch in tqdm(range(num_epochs), desc=f"Épocas - {alias}"):
        n = 0
        for images, boxes, labels in tqdm(dataset, leave=False, desc=f"Imagens - {alias}"):
            optimizer.zero_grad()
            ann = [{'boxes': boxes.to(device), 'labels':labels.to(device)}]
            #annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
            #print(images.unsqueeze(0).to(device), ann)
            #print(images[0], boxes, labels)
            # Forward
            try:
                outputs = model(images.unsqueeze(0).to(device), ann)  # Adiciona batch dimension
            except Exception as e:
                #print(e)
                continue
            # Cálculo da loss (simplificada)
            # O dicionário de saída contém as perdas
            loss = sum(outputs.values())  # Soma todas as perdas para backpropagation
            n+=1
            #print(n, loss.item())
            #clear_count+=1
            #if clear_count % clear_rate == 0:
            #    clear_output()
            #    [print(a) for a in out_epochs]
            
            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            #p_images.update(1)
        
        #out_epochs.append(f"Época {epoch+1}/{num_epochs}, Loss: {loss.item()}")
        #print(f"Época {epoch+1}/{num_epochs}, Loss: {loss.item()}")
        #[print(a) for a in out_epochs]
    return model


In [ ]:
import matplotlib.patches as patches
# Carregar imagem
#image = cv2.imread('/kaggle/input/celulas/imagens/test/BN117.png')
#print(image)
#image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
def plot_detections(image, detections, threshold=0.24):
    image = torch.mean(image.cpu(), dim=1)
    image = image.swapaxes(0,1)
    image = image.swapaxes(1,2)
    fig, ax = plt.subplots(1, figsize=(10, 6))
    ax.imshow(image)
    print(detections)
    for box, score, label in zip(detections[0]["boxes"].to('cpu'), detections[0]["scores"].to('cpu'), detections[0]["labels"].to('cpu')):
        if score > threshold:
            x_min, y_min, x_max, y_max = box
            colors = {0: 'r', 1: 'g', 2:'b'}
            
            rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=2, edgecolor=colors[label.item()], facecolor='none')
            print(box)
            print(rect)
            print(label)
            ax.add_patch(rect)

    plt.show()

#plot_detections(image_tensor, detections)


In [ ]:
import torch
import torchvision.transforms as transforms
import torchvision.models.detection as models
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import os
from PIL import Image
import json
import sys

def avaliacao(model_path, images_path, ann_path, transform, alias=""):
    # Carregar o modelo SSD treinado
    model = models.ssd300_vgg16(pretrained=False)
    model.load_state_dict(torch.load(model_path, map_location="cpu"))
    model.eval()  # Modo de inferência
    
    # Carregar a base de teste (COCO JSON)
    coco_gt = COCO(ann_path)  # Ground Truth
    images_dir = images_path
    
    
    # Lista para armazenar as previsões convertidas para COCO JSON
    coco_results = []
    
    # **4️⃣ Iterar sobre todas as imagens da base de teste**
    for img_id in tqdm(coco_gt.getImgIds(), desc=f"Avaliando - {alias}"):
        img_info = coco_gt.loadImgs(img_id)[0]
        img_path = os.path.join(images_dir, img_info["file_name"])
    
        # Carregar a imagem e aplicar as transformações
        image = Image.open(img_path).convert("RGB")
        image = transform(image).unsqueeze(0)  # Adicionar batch dimension
    
        # Fazer a detecção
        with torch.no_grad():
            outputs = model(image)
    
        # Extrair caixas, rótulos e scores das previsões
        pred_boxes = outputs[0]["boxes"].tolist()
        pred_scores = outputs[0]["scores"].tolist()
        pred_labels = outputs[0]["labels"].tolist()
    
        # **5️⃣ Converter as previsões para o formato COCO**
        for box, score, label in zip(pred_boxes, pred_scores, pred_labels):
            x_min, y_min, x_max, y_max = box
            coco_results.append({
                "image_id": img_id,
                "category_id": label,  # ID da classe
                "bbox": [x_min, y_min, x_max - x_min, y_max - y_min],  # Convertendo para formato COCO (x, y, largura, altura)
                "score": score
            })
    
    # **6️⃣ Salvar as previsões no formato COCO JSON**
    with open(f"predictions_{alias}.json", "w") as f:
        json.dump(coco_results, f, indent=4)

    coco_dt = coco_gt.loadRes(f"predictions_{alias}.json")
    metrics = evaluate_detections(ann_path, f"predictions_{alias}.json")

    with open(f"metrics_{alias}.json", "w") as f:
        f.write(str(metrics))


In [ ]:
import os
import json
import numpy as np
from collections import defaultdict

def load_coco_labels(json_path, tam):
    """Carrega as anotações COCO de um arquivo JSON."""
    with open(json_path, 'r') as f:
        data = json.load(f)
    for a in range(len(data['annotations'])):
        data['annotations'][a]['bbox'] = resize_boxes(data['annotations'][a]['bbox'], (2048, 1536), tam)
    
    annotations = defaultdict(list)
    for ann in data['annotations']:
        image_id = ann['image_id']
        x_min, y_min, width, height = ann['bbox']
        x_max, y_max = x_min + width, y_min + height
        class_id = ann['category_id']
        annotations[image_id].append([class_id, x_min, y_min, x_max, y_max])
    
    return annotations

def load_coco_predictions(json_path, score_threshold=0.0):
    """Carrega predições COCO a partir de um arquivo JSON, filtrando por score."""
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    predictions = defaultdict(list)
    for pred in data:
        if pred['score'] >= score_threshold:
            image_id = pred['image_id']
            category_id = pred['category_id']
            bbox = pred['bbox']
            x_min, y_min, width, height = bbox
            x_max = x_min + width
            y_max = y_min + height
            predictions[image_id].append([category_id, x_min, y_min, x_max, y_max])
    
    return predictions

def calculate_iou(box1, box2):
    """Calcula o IoU entre duas bounding boxes."""
    x1_min, y1_min, x1_max, y1_max = box1[1:]
    x2_min, y2_min, x2_max, y2_max = box2[1:]
    
    xi1, yi1 = max(x1_min, x2_min), max(y1_min, y2_min)
    xi2, yi2 = min(x1_max, x2_max), min(y1_max, y2_max)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area
    
    return inter_area / union_area if union_area > 0 else 0

def compute_ap(recalls, precisions):
    """Calcula o AP (Average Precision) por interpolação da curva precisão-recall."""
    recalls = np.concatenate(([0.0], np.array(recalls), [1.0]))
    precisions = np.concatenate(([0.0], np.array(precisions), [0.0]))
    
    for i in range(len(precisions) - 1, 0, -1):
        precisions[i - 1] = max(precisions[i - 1], precisions[i])
    
    indices = np.where(np.diff(recalls) > 0)[0]
    ap = np.sum((recalls[indices + 1] - recalls[indices]) * precisions[indices + 1])
    return ap

def evaluate_detections(gt_json, pred_json, iou_thresholds=[0.2, 0.5, 0.55, 0.60, 0.65, 0.70, 0.75, 0.80, 0.85, 0.90, 0.95], tam=(300, 300)):
    """Avalia as predições COCO em relação ao ground truth."""
    gt_labels = load_coco_labels(gt_json, tam)
    pred_labels = load_coco_predictions(pred_json)

    
    TP, FP, FN = defaultdict(lambda: defaultdict(int)), defaultdict(lambda: defaultdict(int)), defaultdict(lambda: defaultdict(int))
    
    for img_id in gt_labels.keys():
        gt_bboxes = gt_labels[img_id]
        pred_bboxes = pred_labels.get(img_id, [])
        
        for iou_threshold in iou_thresholds:
            matched = set()
            
            for pred in pred_bboxes:
                best_iou, best_match = 0, None
                for idx, gt in enumerate(gt_bboxes):
                    if gt[0] == pred[0]:
                        iou = calculate_iou(pred, gt)
                        if iou > best_iou:
                            best_iou = iou
                            best_match = idx
                
                if best_iou >= iou_threshold and best_match is not None and best_match not in matched:
                    TP[iou_threshold][pred[0]] += 1
                    matched.add(best_match)
                else:
                    FP[iou_threshold][pred[0]] += 1
            
            for idx, gt in enumerate(gt_bboxes):
                if idx not in matched:
                    FN[iou_threshold][gt[0]] += 1
    
    metrics = {}
    for iou_threshold in iou_thresholds:
        precisions, recalls, aps = {}, {}, {}
        classes = set(TP[iou_threshold].keys()) | set(FN[iou_threshold].keys())
        for cls in classes:
            precisions[cls] = TP[iou_threshold][cls] / (TP[iou_threshold][cls] + FP[iou_threshold][cls]) if (TP[iou_threshold][cls] + FP[iou_threshold][cls]) > 0 else 0
            recalls[cls] = TP[iou_threshold][cls] / (TP[iou_threshold][cls] + FN[iou_threshold][cls]) if (TP[iou_threshold][cls] + FN[iou_threshold][cls]) > 0 else 0
        metrics[iou_threshold] = {"precisions": precisions, "recalls": recalls}
    
    for cls in classes:
        aps[cls] = compute_ap([metrics[iou]['recalls'][cls] for iou in iou_thresholds],
                              [metrics[iou]['precisions'][cls] for iou in iou_thresholds])
    
    for iou_threshold in iou_thresholds:
        metrics[iou_threshold]["aps"] = aps
    
    return metrics


In [ ]:
tamanhos = [(300, 300),
            (640, 640), (720, 720), (1080, 1080)]
num_classes = [
    (1,
    "/kaggle/input/celulas/annotations_vc/annotation_1classe_treino.json",
    "/kaggle/input/celulas/annotations_vc/annotation_1classe_teste.json"),
    (
       2,
       "/kaggle/input/celulas/annotations_vc/annotation_2classe_treino.json",
       "/kaggle/input/celulas/annotations_vc/annotation_2classe_test.json"
    ),
    (3, 
     "/kaggle/input/celulas/annotation/annotation.json", 
     "/kaggle/input/celulas/annotation_test.json"),

    ] # (numero de classes, arquivo de anotacoes de treino, arquivo de anotacoes de teste)

images_train = "/kaggle/input/celulas/annotation/imagens/train"
images_test = "/kaggle/input/celulas/annotation/imagens/test"

for classes, ann_train, ann_test in num_classes:
    for tamanho in tamanhos:
        # if classes == 3 and tamanho in [(300, 300), (640, 640)]:
        #     print("passando", classes, tamanho)
        #     continue
        alias = f"{'_'.join(map(str, tamanho))}_{classes}_classes"
        print(alias)
        # Criar dataset
        transform = transforms.Compose([
            transforms.Resize(tamanho),
            transforms.ToTensor(),
        ])
        dataset = COCODataset(ann_train, images_train, transform=transform, resize=tamanho)
        
        # # treinamento
        model = treinamento(dataset, num_epochs=150, alias=alias)
        # /kaggle/input/modelos-treinados
        PATH = f'/kaggle/input/celulas/ssdresultados/resultados treinamento/{classes}/{tamanho[0]}/modelo_{"_".join(map(str, tamanho))}_{classes}_classes.pth'
        torch.save(model.state_dict(), PATH)
        
        # avaliacao
        
        print("avaliando modelo -", alias)
        avaliacao(PATH, images_test, ann_test, transform, alias=alias)
        metrics = evaluate_detections(ann_test, f"/kaggle/working/predictions_{alias}.json", tam=tamanho)
        # Exibe as métricas
        print("Classe | Precisão | Recall | AP50 | AP75")
        print("--------------------------------------------------")
        t = 0.5
        for cls in metrics[t]['precisions'].keys():
            prec50 = metrics[t]['precisions'][cls]
            rec50 = metrics[t]['recalls'][cls]
            ap50 = metrics[t]['aps'][cls]
            ap5095 = (metrics[t]['aps'][cls] + metrics[0.55]['aps'][cls] + metrics[0.60]['aps'][cls] + metrics[0.65]['aps'][cls] + metrics[0.70]['aps'][cls] + metrics[0.75]['aps'][cls] + metrics[0.80]['aps'][cls] + metrics[0.85]['aps'][cls] + metrics[0.90]['aps'][cls] + metrics[0.95]['aps'][cls]) / 10
            print(f"{cls:^6} | {prec50:.4f} | {rec50:.4f} | {ap50:.4f} | {ap5095:.4f}")